In [ ]:
#Task 1 -Class creation

import requests
import pandas as pd

class DataPoint:
    def __init__(self, api_key):
        self.api_key = api_key

    def find_location_id(self, location_name):
        begin_url = "http://datapoint.metoffice.gov.uk/public/data/"
        resource = "val/wxfcs/all/json/sitelist"
        url = f"{begin_url}{resource}?key={self.api_key}"

        try: 
            response = requests.get(url)
            data = response.json()

            for location in data["Locations"]["Location"]:
                if location["name"] == location_name:
                    return location["id"]

            raise ValueError(f"The location '{location_name}' is not found.")
        
        except requests.exceptions.RequestException as x:
            print("Error in making the API request:", x)
            return None
        except ValueError as x:
            print("Error:", x)
            return None

    def get_forecast_data(self, location_ids):
        begin_url = "http://datapoint.metoffice.gov.uk/public/data/"
        resource = "val/wxfcs/all/json/"
        forecast_days = 5
        forecast_hours = 8  

        every_forecasts = []
        for location_id in location_ids:
            url = f"{begin_url}{resource}{location_id}?res=3hourly&key={self.api_key}"

            try:
                response = requests.get(url)
                data = response.json()

                for day in range(forecast_days):
                    for hour in range(forecast_hours):
                        forecast = data["SiteRep"]["DV"]["Location"]["Period"][day]["Rep"][hour]
                        forecast["Location_ID"] = location_id
                        every_forecasts.append(forecast)
                
            except requests.exceptions.RequestException as x:
                print("Error in making the API request:", x)
                return None
        
        # Convert to DataFrame
        df = pd.DataFrame(every_forecasts)
        return df
